In [29]:
import numpy as np
import pandas as pd
import collections
import datetime
import matplotlib.pyplot as plt
import talib

plt.rcParams['figure.figsize'] = (24, 12)

In [30]:
市值 = pd.read_csv("TEJ_調整後股價資料//市值(百萬元).csv") #TEJ_調整後股價資料 (剛剛TEJ_資料整理 整理的資料夾)
市值['年月日'] = pd.to_datetime(市值['年月日'])
市值 = 市值.set_index('年月日')

In [31]:
收盤價 = pd.read_csv("TEJ_調整後股價資料//收盤價(元).csv")
收盤價['年月日'] = pd.to_datetime(收盤價['年月日'])
收盤價 = 收盤價.set_index('年月日')

In [32]:
市場別 = pd.read_csv("TEJ_調整後股價資料//市場別.csv", low_memory=False)
市場別['年月日'] = pd.to_datetime(市場別['年月日'])
市場別 = 市場別.set_index('年月日')
市場別 = 市場別.apply(lambda x: x.str.strip()) # x.str.strip()把一個字串前後的空白都拿掉

In [69]:
本益比 = pd.read_csv("TEJ_調整後股價資料//本益比-TSE.csv", low_memory=False)
本益比['年月日'] = pd.to_datetime(本益比['年月日'])
本益比 = 本益比.set_index('年月日')

In [72]:
本益比.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4105 entries, 2005-01-03 to 2021-08-20
Columns: 2031 entries, 50 to 916665
dtypes: float64(2031)
memory usage: 63.6 MB


In [71]:
#本益比.convert_objects(convert_numeric=True)
for i in 本益比.columns:
    本益比[i] = pd.to_numeric(本益比[i], errors='coerce')

In [64]:
本益比.columns

Index(['50', '51', '1101', '1102', '1103', '1104', '1107', '1108', '1109',
       '1110',
       ...
       '911613', '911616', '911619', '911622', '911626', '911868', '912000',
       '912398', '913889', '916665'],
      dtype='object', length=2031)

In [65]:
本益比.iloc[4][4]

13.66

In [66]:
print(本益比.iloc[4][4]+5)

18.66


In [67]:
本益比.head()

,50,51,1101,1102,1103,1104,1107,1108,1109,1110,...,911613,911616,911619,911622,911626,911868,912000,912398,913889,916665
年月日,,,,,,,,,,,,,,,,,,,,,
2005-01-03,0.0,NaN,16.59,7.57,14.54,10.39,0.0,7.12,10.48,12.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,0.0,NaN,16.36,7.40,14.35,10.08,0.0,7.04,10.34,12.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,0.0,NaN,16.28,7.30,14.26,10.00,0.0,6.86,10.14,11.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,0.0,NaN,16.12,7.27,14.17,10.12,0.0,6.82,10.48,11.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.0,NaN,15.50,7.13,13.66,10.16,0.0,6.79,10.58,11.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
董監持股 = pd.read_csv("TEJ_董監_經理人持股//董監持股%.csv")
董監持股['年月日'] = pd.to_datetime(董監持股['年月日'])
董監持股 = 董監持股.set_index('年月日')
董監持股.columns = 董監持股.columns.str.strip() #用strip()把column後面的空白拿掉
董監持股 = 董監持股.shift(1).dropna(how = "all", axis = 0) #更改成確定會知道持股數值的日期，因為是下一個月才知道，往後shift(把1/1的值到2/1)，並把第一個row drop掉
董監持股.index = 董監持股.index + datetime.timedelta(14) #把index加上14天(因為下個月15號慈才會公布)

# 預設選股日期
### 定在每個月的16號(15號一定會知道董監事持股)

In [9]:
選股日期 = pd.date_range("2007-01-01", "2021-04-01", freq = "MS") + datetime.timedelta(15) #起始2007-2021 多久一次:MS(Month的第一天)，再加15天
選股前一日 = 選股日期 - datetime.timedelta(1)
選股後一日 = 選股日期 + datetime.timedelta(1)
回測最後一日 = pd.to_datetime("2021-05-15") #4/16加一個月

# 邏輯磚塊

In [10]:
class Condition:
    def __init__(self, data, condition_type, threshold): #condition_type, threshold 條件的種類和門檻
        self.data = data
        self.condition_type = condition_type
        self.threshold = threshold
        
        #每檔股票的門檻值都一樣 or 每檔股票的門檻都不一樣
        if type(threshold) == pd.Series: 
            valid_tickers = list(set(self.data.index) & set(self.threshold.index)) #資料和門檻值都共同有的ticker，把他們共有的tickers拿出來
            self.data = self.data[valid_tickers] #只保留他們倆個都有tickers的資料 和門檻值
            self.threshold = self.threshold[valid_tickers] 
            
    #輸入條件後，要能找出滿足這個條件的股票代號有哪些        
    def build(self, used_tickers = None): #used_tickers:我們要使用的股票代碼(如果使用者沒說要用在哪個股票上，None代表可以用的股票是資料中有的股票代碼)
        if used_tickers is None:
            valid_tickers = list(self.data.index) #等於data裡面有的股票代碼
        else: #如果它有指定要從哪些股票中篩選，必須要是資料裡有的和符合used_tickers(必須兩個都有)
            valid_tickers = list(set(self.data.index) & set(used_tickers))
            
        #根據不同的condition type做處理:一種有排序、一種是跟門檻值比較的(不排序)
        if "rank" in self.condition_type:
            if self.condition_type == "rank_high_pct": # ascending = False:我原本數值比較高的，它的排名比較前面 # pct = True:回傳百分比%(pr值)
                rank_data = self.data[valid_tickers].rank(ascending = False, pct = True, na_option='bottom') 

            elif self.condition_type == "rank_low_pct": # ascending = True我原本數值比較低的，排名比較前面(第一名是比較差的)
                rank_data = self.data[valid_tickers].rank(ascending = True, pct = True, na_option='bottom')

            elif self.condition_type == "rank_high":   # pct = False:回傳名次
                rank_data = self.data[valid_tickers].rank(ascending = False, pct = False, na_option='bottom') # pct = False: 回傳真實的排名

            elif self.condition_type == "rank_low":
                rank_data = self.data[valid_tickers].rank(ascending = True, pct = False, na_option='bottom')
                # rank_data是排名後的data
                
            result = self.data[valid_tickers][rank_data < self.threshold] #可以用的資料中，找排名小於我們門檻值的
        
        else: #純粹跟門檻值比較的
            if self.condition_type == "compare_high":
                result = self.data[valid_tickers][self.data > self.threshold] #大於門檻值

            elif self.condition_type == "compare_low":
                result = self.data[valid_tickers][self.data < self.threshold] #小於門檻值

            elif self.condition_type == "equal":
                result = self.data[valid_tickers][self.data == self.threshold]
            
        return list(result.index) #回傳結果(哪些tickers滿足這些條件)
    

In [11]:
class ConditionAnd: #把兩個條件結合起來變成一個新的條件
    def __init__(self, conditions):
        self.conditions = conditions #把傳進來的conditions的list(裡面包含很多condition)回傳給自己
    
    def build(self, used_tickers = None):
        tickers = None
        for condition in self.conditions: #把每個condition run一遍
            result  = condition.build(used_tickers) #取得符合這個condition的tickers，再把這些tickers做交集的邏輯運算，得到新的可以的tickers
            
            if tickers is None:
                tickers = result
            else:
                tickers = list(set(tickers) & set(result))  #做condition的交集
                
        return tickers
    
class ConditionOR: #把兩個條件or起來變成一個新的條件
    def __init__(self, conditions):
        self.conditions = conditions
    
    def build(self, used_tickers = None):
        tickers = None
        for condition in self.conditions:
            result  = condition.build(used_tickers)
            
            if tickers is None:
                tickers = result
            else:
                tickers = list(set(tickers) | set(result)) #做condition的聯集
                
        return tickers

In [12]:
#不同層級的條件合成(上下集合的關係)

class ConditionBlocks:
    def __init__(self):
        self.condition_blocks = []  #初始是一個空的list
        
    def add_condition(self, condition):
        self.condition_blocks.append(condition)
        
    def build(self):
        tickers = None
        for condition in self.condition_blocks:
            if tickers is None:
                tickers = condition.build() #拿到tickers
            else:
                tickers = condition.build(tickers) #前面條件篩選過後的再篩選
                
        return tickers

# 回測

### 上市股票市值前50%，董監事持股增加(同個層級)，10日均>20日均，本益比低於15倍

In [45]:
本益比.head()

,50,51,1101,1102,1103,1104,1107,1108,1109,1110,...,911613,911616,911619,911622,911626,911868,912000,912398,913889,916665
年月日,,,,,,,,,,,,,,,,,,,,,
2005-01-03,0.0,NaN,16.59,7.57,14.54,10.39,0.0,7.12,10.48,12.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,0.0,NaN,16.36,7.40,14.35,10.08,0.0,7.04,10.34,12.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,0.0,NaN,16.28,7.30,14.26,10.00,0.0,6.86,10.14,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,0.0,NaN,16.12,7.27,14.17,10.12,0.0,6.82,10.48,11.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.0,NaN,15.50,7.13,13.66,10.16,0.0,6.79,10.58,11.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
N = 2
K = 10 #均線長度
fee = 1.425/1000 * 0.6 #手續費
tax = 3/1000 #交易稅
retrun_series = pd.Series([1], [選股日期[0]]) #儲存回測結果
tickers_record = [] #儲存選到的股票

for i in range(len(選股日期)):
    日期 = 選股日期[i]
    #print(日期)
    
    # 準備資料
    當期市場別 = 市場別.loc[:日期].iloc[-1].dropna() # loc[:日期].iloc[-1]選到這個日期前所有可以看到資料中的最後一筆，然後drop掉na
    當期市值 = 市值.loc[:日期].iloc[-1].dropna()
    均線 = 收盤價.loc[:日期].iloc[-K:].mean().dropna() # k日均線(取當前可以看到資料的最後k筆的平均)
    均線2 = 收盤價.loc[:日期].iloc[-K*2:].mean().dropna()
    前N期董監持股 = 董監持股.loc[:日期][-N:] # 倒數兩次的持股比例
    董監持股成長率 = 前N期董監持股.iloc[-1] / 前N期董監持股.iloc[-2] - 1
    董監持股成長率 = 董監持股成長率.dropna() #把沒有值的drop掉
    #本益比 = 本益比.dropna()
    當期本益比 = 本益比.loc[:日期].iloc[-1].dropna()  # .loc[:日期].iloc[-1] 代表式取得資料的最後一筆本益比數值(沒加的話會不知道是取哪一天的)
    資料齊全的股票 = list(set(董監持股成長率.index) & set(當期市值.index) & set(當期市場別.index) & set(本益比.index))
    
    # 增加篩選條件
    condition_1 = Condition(當期市場別, "equal", "TSE")  #條件: 上市股票
    condition_2 = Condition(當期市值, "rank_high_pct", 0.5) #條件: 市值前50%
    condition_3 = Condition(董監持股成長率, "compare_high", 0)#條件: 董監持股成長率>0
    condition_4 = Condition(均線, "compare_high", 均線2) #條件: 10日均>20日
    condition_5 = Condition(當期本益比, "rank_high_pct", 0.5) #條件: 本益比<15倍
    
    
    condition_blocks = ConditionBlocks()
    condition_blocks.add_condition(condition_1)
    condition_blocks.add_condition(ConditionAnd([condition_2, condition_3, condition_4, condition_5])) #2、3、4、5條件都成立
    tickers = condition_blocks.build()
    tickers_record.extend(tickers)  #選好了要交易的股票了
    display(tickers)
    
    if i != len(選股日期) - 1:
        收盤價資料 = 收盤價.loc[選股後一日[i]:選股日期[i+1]] #如果不是最後一次選股日期，收盤價=選股後一日到下一次選股

    else:
        收盤價資料 = 收盤價.loc[選股後一日[i]:回測最後一日]

    if len(tickers) != 0:  #有選出股票的情況下
        收盤價資料 = 收盤價資料[tickers]    
        選股日報酬 = 收盤價資料.pct_change().dropna(how = "all", axis = 0).fillna(0) #NA值用0補起來
        
        if len(選股日報酬) != 0:  # 不是選股日報酬=0(選到後隔天下市)
            選股日報酬.iloc[0] = 選股日報酬.iloc[0] - fee
            選股日報酬.iloc[-1] = 選股日報酬.iloc[-1] - fee - tax  #扣手續費和稅
            區間累積報酬 = (選股日報酬 + 1).cumprod().mean(axis = 1)  # 累乘後再取平均
            
    else:
        區間累積報酬 = pd.Series(1, 收盤價資料.index) #沒選到股票的話是 一串1
        
    retrun_series = pd.concat([retrun_series, 區間累積報酬*retrun_series.iloc[-1]]) #接上RETURN SERIES後面，因為是累積報酬的序列，要乘上累積報酬的最後一個值
            
tickers_record = dict(collections.Counter(tickers_record)) #算說每個ticker出現的次數，轉換成dictionary
tickers_record = dict(sorted(tickers_record.items(), key=lambda x: x[1], reverse=True)) #再去把dictionary做排序，次數越高在dictionary的順序越前面

['2393', '2457', '2432', '2450', '2387', '2344', '3009', '3063']

['9908', '2520', '2448', '1503', '2912']

['3014', '2448', '2489', '2027']

['9910', '2347', '3702', '2448', '2106', '2432', '2888', '1507', '2439']

['2393', '1606', '2391', '2499', '2105', '2538', '2888']

...

['2332', '1536', '6581', '3380', '3035', '2439']

['2511',
 '1532',
 '3665',
 '2207',
 '1225',
 '1707',
 '1515',
 '3454',
 '6196',
 '6230',
 '3380',
 '6271',
 '3035']

['9802', '6669', '6706', '1216']

['1532', '8926', '1225', '2409', '6581', '3380', '2489']

['2031',
 '1532',
 '2617',
 '8462',
 '5007',
 '2481',
 '8016',
 '1225',
 '2409',
 '9921',
 '1402',
 '1519',
 '2436',
 '3545',
 '9942',
 '1310',
 '2489']

In [60]:
本益比.head(4)

,50,51,1101,1102,1103,1104,1107,1108,1109,1110,...,911613,911616,911619,911622,911626,911868,912000,912398,913889,916665
年月日,,,,,,,,,,,,,,,,,,,,,


ValueError: could not convert string to float: '          -'